In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('BX-Book-Ratings.csv', engine='python')
df2 = pd.read_csv('BX-Books.csv', engine='python')
df3 = pd.read_csv('BX-Users.csv', engine='python')
df3 = df3.drop('user_id', axis=1)
df = df1.merge(df2, on = 'isbn')
df = pd.concat([df, df3], axis = 1)
df = df.fillna(0, axis = 1)
df = df[:10000]
df

In [ ]:
from sklearn.model_selection import train_test_split 
n_users = df.year_of_publication.unique().shape[0]
n_rating = df.Age.unique().shape[0]
train_data, test_data = train_test_split(df, test_size=0.25, random_state=42)

In [ ]:
train_data_matrix = np.zeros((n_pub, n_age))
for line in train_data.itertuples():
    train_data_matrix[int(line.year_of_publication), int(line.Age)] = line[3]  
train_data_matrix

In [ ]:
test_data_matrix = np.zeros((n_users, n_rating))
for line in test_data.itertuples():
    #[user_id index, movie_id index] = given rating.
    test_data_matrix[line[1]-1, line[2]-1] = line[3]
test_data_matrix

In [ ]:
from sklearn.metrics import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
rating_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')
mean_user_rating = train_data_matrix.mean(axis=1)[:, np.newaxis] 
ratings_diff = (train_data_matrix - mean_user_rating) 
user_pred = mean_user_rating + user_similarity.dot(ratings_diff) / np.array([np.abs(user_similarity).sum(axis=1)]).T
user_pred

In [ ]:
rent_pred = train_data_matrix.dot(rating_similarity) / np.array([np.abs(rating_similarity).sum(axis=1)])
rent_pred

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(pred, test):
    pred = pred[test.nonzero()].flatten() 
    test = test[test.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, test))

In [ ]:
rmse(user_pred, test_data_matrix)

In [ ]:
rmse(rent_pred, test_data_matrix)